# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [43]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from datasets import load_dataset
import torch
device = torch.device('cuda')
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)
model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [44]:
# load dataset
dataset = load_dataset("imdb", split="test[:5]")
print(dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 5
})


In [45]:
def preprocess_data(examples):
    return tokenizer(examples["text"], padding = True, truncation = True)
tokenized_dataset = dataset.map(preprocess_data, batched = True)
print(tokenized_dataset)

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 5
})


In [47]:
def evaluate_model(model, dataset, tokenizer):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for example in dataset:
            inputs = tokenizer(example["text"], return_tensors="pt", truncation=True, padding=True)
            inputs.to(device)
            outputs = model(**inputs)
            prediction = torch.argmax(outputs.logits, dim=1)
            labels = example["label"]
            correct += int(prediction == labels)
            total += 1
        return correct/total

In [48]:
original_model_accuracy = evaluate_model(model, tokenized_dataset, tokenizer)
print(original_model_accuracy)

1.0


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [49]:
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_int8_training
from bitsandbytes.optim import AdamW
quantized_model = prepare_model_for_int8_training(model)
train_dataset = load_dataset("imdb", split="train[:10]")
eval_dataset = load_dataset("imdb", split="test[:10]")
tokenized_train_dataset = train_dataset.map(preprocess_data, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_data, batched=True)
print(tokenized_train_dataset)
print(tokenized_eval_dataset)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10
})
Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10
})


In [51]:
lora_config = LoraConfig(
    target_modules=["out_lin"],
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1
)

In [52]:
from transformers import TrainingArguments, Trainer
peft_quantized_model = get_peft_model(quantized_model, lora_config)
print(peft_quantized_model)
for parameter in peft_quantized_model.classifier.parameters():
    parameter.require_grad = True
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate= 0.002,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs"
)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(in_features=768, out_features=768, bias=True)
                (k_lin): Linear(in_features=768, out_features=768, bias=True)
                (v_lin): Linear(in_features=768, out_features=768, bias=True)
                (out_lin): Linear(
                  in_features=768, out_features=768, bias=True
           

In [53]:
optimizer = AdamW(peft_quantized_model.parameters(), lr=0.002)
trainer = Trainer(
    model=peft_quantized_model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    optimizers=(optimizer,None)
)
trainer.train()
trainer.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.002579
2,No log,0.000029
3,No log,0.000004


Checkpoint destination directory ./results/checkpoint-2 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-4 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-6 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'eval_loss': 3.969660156144528e-06,
 'eval_runtime': 0.1557,
 'eval_samples_per_second': 64.213,
 'eval_steps_per_second': 12.843,
 'epoch': 3.0}

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [54]:
# Saving the model
peft_quantized_model.save_pretrained("/tmp/pqt")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [55]:
# use different dataset for testing
# Load the full IMDB test dataset
test_dataset = load_dataset("imdb", split="test[50:60]")

shuffled_test_dataset = test_dataset.shuffle(seed=5)

tokenized_shuffled_testdata = shuffled_test_dataset.map(preprocess_data, batched=True)

print(tokenized_shuffled_testdata)

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10
})


In [56]:
original_accuracy = evaluate_model(model, tokenized_shuffled_testdata, tokenizer)
print(original_accuracy)

1.0


In [57]:
peft_accuracy = evaluate_model(peft_quantized_model, tokenized_shuffled_testdata, tokenizer)
print(peft_accuracy)

1.0
